# Analysis of Orbits in Fourier Space

Here we simulate Dan's Machine Learning system with rebound and then do a basic Fourier analysis. 

In [1]:
import rebound
import numpy as np
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
#%matplotlib inline

Here we set up the outer solar system with the possibility to inflate the masses. Such a system is known to go unstable on short timescales (e.g. Chambers, 1999). The simulation stops once there is a close encounter. 

In [2]:
def problem(inner_massfac,output_N,tmax,seed):
    sim = rebound.Simulation()
    sim.integrator = "whfast"
    sim.dt = 1e-4*2*np.pi
    
    #add bodies
    m_planet = 5*3e-6    #5-earth masses
    a_planet = 1
    rhill = a_planet*(m_planet/3.)**(1./3.)
    e = 1e-6
    
    np.random.seed(seed)
    
    sim.add(m=1.)
    sim.add(a=a_planet - 52.63*rhill,m=m_planet*inner_massfac,e=e,inc=1e-4,f=np.random.random()*np.pi*2)
    sim.add(a=a_planet,m=m_planet,e=e,inc=2e-4,f=np.random.random()*np.pi*2)
    sim.add(a=a_planet + 8*rhill,m=m_planet,e=e,inc=1e-5,f=np.random.random()*np.pi*2)
    sim.N_active = sim.N
    
    sim.move_to_com()
    E0 = sim.calculate_energy()
    #sim.init_megno()
    
    #condition to stop the simulation
    m_neptune = 5e-5
    sim.exit_min_distance = a_planet*(m_neptune/3.)**(1./3.)
    
    #integrate and output
    output_name = "Fourier/data/DTsys_t"+str('{0:.1e}'.format(tmax))+"_innerMF"+str(inner_massfac)+"_sd"+str(seed)+".csv"
    output_info = "Fourier/data/DTsys_t"+str('{0:.1e}'.format(tmax))+"_innerMF"+str(inner_massfac)+"_sd"+str(seed)+"_info.txt"
    start_time = time.time()
    times = np.linspace(0,tmax*2*np.pi,output_N)
    print 'starting simulation for '+output_name
    with open(output_name, 'w') as f:
        try:
            for t in times:
                sim.integrate(t)
                f.write(str(sim.t/(2*np.pi)))
                #f.write(','+str(sim.calculate_lyapunov()))
                outs = []
                for p in sim.particles[1:sim.N - sim.N_var]:
                    outs += [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
                for out in outs:
                    f.write(',{0}'.format(out))
                f.write('\n')
            f.close()
            o = open(output_info,"w")
            o.write('Simulation complete. Final energy error is: '+str(abs((E0 - sim.calculate_energy())/E0))+".")
            o.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            o.close()
            print 'Simulation '+output_name+' complete. Final energy error is:', abs((E0 - sim.calculate_energy())/E0)
        except rebound.Encounter as error:
            f.write(str(sim.t/(2*np.pi)))
            #f.write(','+str(sim.calculate_lyapunov()))
            outs = []
            for p in sim.particles[1:sim.N - sim.N_var]:
                outs += [p.a, p.P, p.e, p.pomega, p.inc, p.Omega, p.f]
            outs += []
            for out in outs:
                f.write(',{0}'.format(out))
            f.write('\n')
            f.close()
            o = open(output_info,"w")
            o.write("Close encounter, simulation "+output_name+" stopping.")
            o.write('\nTotal Elapsed time is: '+str(time.time() - start_time)+' seconds.')
            o.close()
            print "Close encounter, simulation "+output_name+" stopping."

Choose the parameters you will search over.

In [ ]:
n_runs = 10             #number of simulations you want to run
max_inner_massfac = 1   #max mass factor to increase inner planet by (60 = Jupiter)
n_outputs = 5e4         #number of outputted data points, important for FFT
tmax = 1e6              #simulation time

**Warning!** Executing the code below will take a long time!

In [ ]:
#Get inputs
massfac = np.round(np.logspace(0,np.log10(max_inner_massfac),n_runs),2)

#temp, consistency check - two of every run to see if fourier specs agree
#massfac = np.concatenate((massfac,massfac))
#n_runs *= 2

params = zip(massfac,n_outputs*np.ones(n_runs),tmax*np.ones(n_runs),(1000*np.random.random(n_runs)).astype("int"))

#Run the problem
import multiprocessing as mp
def execute(pars):
    problem(pars[0],pars[1],pars[2],pars[3])

if __name__== '__main__':
    pool = mp.Pool(processes=n_runs)
    pool.map(execute, params)
    pool.close()
    pool.join()

# Functions
Here are the functions we need to create to do a Fourier analysis

In [ ]:
#extra variables that we might want to plot
def get_var(data,i,var):
    if var == "LCE":
        return data["LCE"]
    if var == "z":
        e = data["e"+i].values
        pomega = data["pomega"+i].values
        return e*np.exp(pomega*1j)
    if var == "h":
        e = data["e"+i].values
        pomega = data["pomega"+i].values
        return e*np.cos(pomega)
    if var == "k":
        e = data["e"+i].values
        pomega = data["pomega"+i].values
        return e*np.sin(pomega)
    else:
        return data[var+i].values

In [ ]:
#1-D frequency map
import scipy.signal as signal
def fourier(data,var,tmax,N_outputs,output_name):        
    N_freq = 200
    t=data["Time"].values
    window = np.hanning(N_outputs)
    freq = np.fft.fftfreq(N_outputs, tmax/N_outputs)
    plt.figure(figsize=(12,8))
    names = ['Planet 1','Planet 2','Planet 3','Planet 4']
    for i,num in enumerate(['1', '2', '3']):
        y = get_var(data,num,var)
        power = np.fft.fft(y)/N_outputs
        plt.subplot(2, 1, 1)
        plt.plot(1./freq,power,label=names[i],alpha=1-0.2*i)
        plt.subplot(2, 1, 2)
        plt.plot(t,y,label=num,alpha=1-0.2*i)

    plt.subplots_adjust(hspace=0.4)
    plt.subplot(2, 1, 1)
    plt.ylabel("power of: "+var)
    plt.legend(loc="upper left")
    plt.xlabel('frequencies')
    plt.xscale('log')
    plt.subplot(2, 1, 2)
    plt.ylabel(var)
    plt.xlabel('time')
    if var == "LCE":
        plt.yscale('log')
    out = output_name.split(".csv")[0].split("data")[1]
    plt.savefig("Fourier/images"+out+"_fft-"+var+".png")
    plt.clf()

# Analyze Batch
Let's batch analyze our data now (note you may have to change the file_location variable).

In [ ]:
#If just wanting to batch analyze .csv files
file_location = 'Fourier/data/DT_1e6/*.csv'  #Location of where data is stored. 
var = "z"                        #choices are h,k,z or any of the orbital elements (e.g. a,P,e,pomega,etc.)

In [ ]:
import glob
files = glob.glob(file_location)
columns = ['Time']
for i in ['1', '2', '3']:
    columns += ['a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i]

for f in files:
    data = pd.read_csv(f,header=None,names=columns)
    tmax = float(f.split("_")[-3].split("t")[1])
    N_outputs = len(data.index)
    fourier(data,var,tmax,N_outputs,f)

# Run/Analyze single

In [ ]:
#single run
problem(1,1000,1e3)

In [ ]:
%matplotlib inline
f = "Fourier/data/DTsys_t1.0e+06_innerMF1_sd198.csv"
var = "z"  
columns = ['Time']
for i in ['1', '2', '3', '4']:
    columns += ['a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i]
    
data = pd.read_csv(f,header=None,names=columns)
data.head()
tmax = float(f.split("_")[-3].split("t")[1])
N_outputs = len(data.index)
fourier(data,var,tmax,N_outputs,f)